In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/week-7-ann-heart-failure-prediction/sample.csv
/kaggle/input/week-7-ann-heart-failure-prediction/train.csv
/kaggle/input/week-7-ann-heart-failure-prediction/test.csv


In [29]:
train_df = pd.read_csv('/kaggle/input/week-7-ann-heart-failure-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/week-7-ann-heart-failure-prediction/test.csv')

train_df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,83327,18995,2,162,83.0,120,80,1,1,0,0,0,1
1,86196,17319,1,158,64.0,120,80,1,1,0,0,1,0
2,59158,19017,1,165,95.0,160,100,2,1,0,0,1,1
3,16399,20388,1,164,83.0,150,100,1,1,0,0,1,1
4,29470,18236,1,156,52.0,100,67,1,1,0,0,0,0


In [30]:
from sklearn.preprocessing import StandardScaler

# Step 1: define the preprocessing function (no scaling yet)
def preprocess(df):
    df = df.copy()
    
    df['age_years'] = (df['age'] / 365).astype(int)
    df.drop('age', axis=1, inplace=True)

    df['bmi'] = df['weight'] / ((df['height'] / 100) ** 2)

    df['bp_high'] = ((df['ap_hi'] >= 140) | (df['ap_lo'] >= 90)).astype(int)
    df['bp_low'] = ((df['ap_hi'] <= 90) | (df['ap_lo'] <= 60)).astype(int)

    df['smoke_alco'] = df['smoke'] & df['alco']
    df['healthy_lifestyle'] = ((df['smoke'] == 0) & 
                               (df['alco'] == 0) & 
                               (df['active'] == 1)).astype(int)
    
    return df

# Step 2: preprocess both
train_df = preprocess(train_df)
test_df = preprocess(test_df)

# Step 3: scale selected columns using StandardScaler
columns_to_scale = ['age_years', 'height', 'weight', 'bmi', 'ap_hi', 'ap_lo']
scaler = StandardScaler()

# Fit only on train, transform both
train_df[columns_to_scale] = scaler.fit_transform(train_df[columns_to_scale])
test_df[columns_to_scale] = scaler.transform(test_df[columns_to_scale])

In [31]:
test_df.head()

,id,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,age_years,bmi,bp_high,bp_low,smoke_alco,healthy_lifestyle
0,26681,1,-1.135796,-1.023103,-0.055968,-0.060952,1,1,0,0,1,0.024442,-0.455301,0,0,0,1
1,58585,1,-0.528034,-1.057850,0.005932,-0.035121,1,1,0,0,1,0.616852,-0.733935,1,0,0,1
2,54339,2,1.295251,0.957492,-0.055968,-0.086783,2,1,0,0,1,-1.752787,0.188011,0,0,0,1
3,17273,2,1.538356,-0.849366,-0.055968,-0.035121,1,1,0,0,1,-0.271763,-1.261851,1,0,0,1
4,25420,1,0.322832,0.471030,-0.055968,-0.086783,1,1,0,0,1,-0.567967,0.238099,0,0,0,1


In [32]:
train_df.head()

,id,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,age_years,bmi,bp_high,bp_low,smoke_alco,healthy_lifestyle
0,83327,2,-0.284930,0.610019,-0.055968,-0.086783,1,1,0,0,0,1,-0.123660,0.656711,0,0,0,0
1,86196,1,-0.771139,-0.710377,-0.055968,-0.086783,1,1,0,0,1,0,-0.864172,-0.314112,0,0,0,1
2,59158,1,0.079728,1.443954,0.191632,0.016541,2,1,0,0,1,1,-0.123660,1.186445,1,0,0,1
3,16399,1,-0.041825,0.610019,0.129732,0.016541,1,1,0,0,1,1,0.320647,0.532441,1,0,0,1
4,29470,1,-1.014244,-1.544312,-0.179769,-0.153944,1,1,0,0,0,0,-0.567967,-1.006144,0,0,0,0


In [33]:
train_df = train_df.drop(columns=['id'])  # Drop from train
test_ids = test_df['id'].copy()
test_df = test_df.drop(columns=['id'])

In [34]:
X_train = train_df.drop(columns=['cardio'])
y_train = train_df['cardio']

X_test = test_df

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [36]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

In [37]:
from torch.utils.data import TensorDataset, DataLoader

# Train dataset and loader — with labels
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Test loader — just the features, no labels
test_loader = DataLoader(X_test_tensor, batch_size=64)

# Define model

In [38]:
class HeartFailureModel(nn.Module):
    def __init__(self, input_size):
        super(HeartFailureModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

input_size = X_train.shape[1]
model = HeartFailureModel(input_size)

In [39]:
criterion = nn.BCELoss()  # Binary Cross Entropy for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [40]:
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/25, Loss: 0.5774
Epoch 2/25, Loss: 0.5659
Epoch 3/25, Loss: 0.5623
Epoch 4/25, Loss: 0.5591
Epoch 5/25, Loss: 0.5570
Epoch 6/25, Loss: 0.5535
Epoch 7/25, Loss: 0.5509
Epoch 8/25, Loss: 0.5517
Epoch 9/25, Loss: 0.5473
Epoch 10/25, Loss: 0.5465
Epoch 11/25, Loss: 0.5465
Epoch 12/25, Loss: 0.5446
Epoch 13/25, Loss: 0.5452
Epoch 14/25, Loss: 0.5450
Epoch 15/25, Loss: 0.5448
Epoch 16/25, Loss: 0.5441
Epoch 17/25, Loss: 0.5440
Epoch 18/25, Loss: 0.5440
Epoch 19/25, Loss: 0.5443
Epoch 20/25, Loss: 0.5433
Epoch 21/25, Loss: 0.5427
Epoch 22/25, Loss: 0.5422
Epoch 23/25, Loss: 0.5419
Epoch 24/25, Loss: 0.5416
Epoch 25/25, Loss: 0.5440


In [41]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_class = (y_pred >= 0.5).float()
    # accuracy = (y_pred_class == y_test_tensor).float().mean()
    # print(f"Test Accuracy: {accuracy:.4f}")

In [43]:
# Match predictions back to ids
results_df = pd.DataFrame({
    'id': test_ids.values,
    'prediction': y_pred_class.numpy().flatten().astype(int)
})

In [44]:

# Save to CSV
results_df.to_csv('submission.csv', index=False)